先把-1变成1 并记录位置，然后跑一边dijkstra记录source到各点最短路，如果到
destination最短路大于target则意味着最短的最短路（后面1还要变大）都大于target，直接退出。
跑第二次dijkstra，对于每个可修改（原来是-1权值）的点依据使得到destination最短路==target进行修改（1u + x + d0e - d0v = t -> x = t - d1u - d0e + d0v）如果修改值>原来权值则应用修改。最后跑完后到destination最短路为target则返回edges 否则空

In [ ]:
# 参考灵神视频讲解
# https://leetcode.cn/problems/modify-graph-edge-weights/solutions/2278296/xiang-xi-fen-xi-liang-ci-dijkstrachou-mi-gv1m/
from ast import List
from collections import defaultdict
from heapq import heappop, heappush
from math import inf


class Solution:

    def modifiedGraphEdges(self, n: int, edges: List[List[int]], source: int, destination: int, target: int) -> List[List[int]]:
        # 1. 将 -1 -> 1 并记录
        todo = {}
        adj = [[] for _ in range(n)]
        for i, (u, v, w) in enumerate(edges):
            if w == -1:
                w = edges[i][2] = 1
                todo[(u, v)] = todo[(v, u)] = i
            adj[u].append((v, w))
            adj[v].append((u, w))
        # 2.第一次dijkstra
        pq = [(0, source)]
        costs0 = defaultdict(lambda: inf)
        costs0[source] = 0
        best = set()
        while pq:
            cost, u = heappop(pq)
            best.add(u)
            for v, w in adj[u]:
                if v not in best and (ncost := cost + w) < costs0[v]:
                    costs0[v] = ncost
                    heappush(pq, (ncost, v))
        if costs0[destination] > target: return []  # 最短路只会更大
        if costs0[destination] == target: return edges
        # 3.第二次dijkstra
        pq = [(0, source)]
        costs1 = defaultdict(lambda: inf)
        costs1[source] = 0
        best = set()
        while pq:
            cost, u = heappop(pq)
            best.add(u)
            for v, w in adj[u]:
                if v not in best:
                    if (u, v) in todo:
                        # d1u + x + d0e - d0v = t -> x = t - d1u - d0e + d0v
                        # x: 为了使得最短路==target的(u,v)权值的目标修改值
                        wt = target - costs1[u] - costs0[destination] + costs0[v]
                        if wt > w:  # 只允许更大 当前为1
                            w = edges[todo[(u, v)]][2] = wt
                    if (ncost := cost + w) < costs1[v]:
                        costs1[v] = ncost
                        heappush(pq, (ncost, v))
        return edges if costs1[destination] == target else []